# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [4]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [5]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution
lst = ['play', 'ground']
print 'ground' in lst

m = matrix(QQ, [[1,2],[4,2],[11,3]])
print 'm = ',m
x = []
for v in m:
    x.append(v)
    
x = matrix(QQ, x)
print 'x = ',x
 
pr = Permutations(20).random_element()
print pr, pr[4], pr[7]


v = vector(QQ, [2,3,4])
print 'len = ',len(v)

min(5, 44)

l = [1,2,3,4,5]
print l [2:4]

 solution = [None, False, False, True]
True
m =  [ 1  2]
[ 4  2]
[11  3]
x =  [ 1  2]
[ 4  2]
[11  3]
[6, 7, 12, 16, 10, 9, 20, 11, 1, 15, 13, 8, 17, 19, 2, 18, 14, 3, 4, 5] 10 11
len =  3
[3, 4]


In [6]:
# TODO: check correctness etc
def get_at(tup, a, l):
    """Given 2*l elements flattened into a tuple, pick 1 out of each pair according to array of bits."""
    assert len(tup) == 2*l
    assert len(a) == l
    for bit in a:
        assert bit in (0, 1)

    # tup structure is: $(t_{0,0}, t_{0,1}, t_{1,0}, t_{1,1}, t_{2,0}, t_{2,1})$
    return tuple(tup[2*i + a[i]] for i in range(l))

assert get_at((3, 13, 85, 95), (0, 1), 2) == (3, 95)

In [65]:
 
def tuples_fixed_proj(l, proj_ind, proj_val):
#    h = randint(1,4000)
#    if (h == 5):
#        print '==================== DEBUGGING ========== a,g = ', proj_ind, proj_val
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
#        if (h == 5):
#            print 'modified x to ', x
        tuples.append(tuple(x[:]))
    return tuples     

# Hack from correctness and server security (!) 
# strength = 0 is consistent with standard LP. strength = 1 attempts to better sort things out   

def add_no_id_a(solver, p0, q1, strength = 0):
    
    if strength == -1:
        return
    
    for i in range(3):                              
        for a in itertools.product(range(2), repeat = l):
            for j in range(3):
                if (j > i):
                    # this is relaxed
                    if strength == 0:
                        solver.add_constraint(sum(q1[(i, a, zeros, b)] for b in range(2)) + sum(q1[(j, a, zeros, b)] 
                                                 for b in range(2)) <= 1)    
                    elif strength == 1:
                        for b1 in range(2):
                            for b2 in range(2):
                                solver.add_constraint(p0[(i, a, zeros, b1)] + p0[(i, a, zeros, b2)] <= 1)
                                
# TODO: change the name to something more informative.
# Hack #2 from correctness: For every i,a,b with p^i_a > 0, there exists g so that (i,a,g,b) > 0.
# This requires some trial and error. Otherwise, one xi at least remains without TO-input options
# This will eventually not be part of the client constraints. Currently used to `direct' the soutions.
    

# Another hack from correctness. For all b, there exists at least one value of t, so 
# that every a reading this g will output b. Thus, the resulting sum of probabilities for
# that g is 1. There may be additional g's (the g results from some V assigned to server for each output
# value).                


def get_rand_tup(len):
    return tuple([sage.misc.prandom.choice(range(2)) for i in range(len)])
                

# i = -1 , search for an index
# i = -2 : auxiliary for relaxed mode
# [1,0,0]
# [0,1,0]
# [0,0,1]
# [0,0,0]

def safe_insert(val_l, ind_l, (i,ind), v):
    to_search = []
    if i >= 0:
        to_search = [i]
    elif i == -1:
        to_search = [j for j in range(3)]
    elif i == -2:
        val_l[ind_l.index(ind)] = v
        return
    
    for j in to_search:
        if (j,ind) in ind_l:
            val_l[ind_l.index((j,ind))] = v


def find_set_server_vars(l, p0):
    try:
        print 'How much is server variable space limited by this solution & correctness?'
        print 'List = ',p0
        p1 = []
        for i in range(3):                              
            for t in itertools.product(range(2), repeat = 2*l):
                p1.append((i,t))

        for (y,(a,g,b)) in p0:
            for j in range(3):
                if ((b == 0) and (i == j)) or ((b == 1) and (i != j)):
                    all = tuples_fixed_proj(l, a, g)
                    for t in all:
                        try:
                            p1.remove((j, t))
                        except Exception as e:
                            pass

        print 'number of remaining server values =',len(p1)
        if (len(p1) > 0):
            print 'server variables are ',p1
    except: 'Uncaught exception!'    


def permute_rows(m, b):
    
    nrows = m.nrows()
    perm = Permutations(nrows).random_element()
    pm = [row for row in m]
    pb = [v for v in b]
    
    new_m = matrix(QQ, [pm[perm[i] - 1] for i in range(nrows)])
    
    new_b = vector(QQ, [pb[perm[i] - 1] for i in range(nrows)])
    
    return (new_m, new_b)
    
    
    
def get_full_rank_sub(m, b):
   
    nrows = m.nrows()
    new_mat = []
    new_b = []
    
 #   print 'into get_full_rank_sub',m.nrows(), m.ncols()
    
    for (i,row) in enumerate(m):
        so_far = (matrix(QQ, new_mat)).transpose()
        try:
            so_far\row
        except Exception as e:
#            print 'at full rank sub Exception = ',e,i
            new_mat.append(row)
            new_b.append(b[i])
        
    new_mat = matrix(QQ, new_mat)
    new_b = vector(new_b)
    
    return (new_mat, new_b)


def print_matrix(m):

    if m.ncols < 37:
        print m
        return
    
    per_row = 30
    for row in m:
        n = ceil(QQ(m.nrows())/per_row)
        for i in range(n):
            ran = min(per_row, m.nrows() - i * per_row)
            print [row[i * per_row + j] for j in range(ran)]
        print '\n'    
          

def non_zeros(m):            
    return sum(sum(1 for j in row if abs(j) != 0) for row in m)        
        
        
def get_inv_sub_matrix(m, b):
    
    (new_mat, new_b) =  get_full_rank_sub(m, b)
    
    ncols = new_mat.ncols()
    print 'new_mat rank /  ',new_mat.rank(),'(',new_mat.nrows(),',',new_mat.ncols(),')'
    
    new_mat_tr = new_mat.transpose()
    b_dum = vector(QQ, [0 for i in range(ncols)]) 
    
    
    (full_sub, b_dum) = permute_rows(new_mat_tr, b_dum)
    (full_sub, b_dum) = get_full_rank_sub(full_sub, b_dum)
    
    inv_sub = full_sub.inverse()
    
    print 'x = A^{-1}*b',
 #   print full_sub
    
    det_A = full_sub.determinant()
    print '|A| = ', det_A
    # Finding independent columns
    print 'Sparsity parameters',non_zeros(full_sub),' => ',non_zeros(inv_sub)
    
    x = inv_sub * new_b
    print 'x = ',x
    print 'len(x) = ',len(x)
    
    if (full_sub.nrows() < 50 and abs(det_A) > 1) or (full_sub.ncols() < 40):
        print print_matrix(inv_sub)
    
    return x

def test_linear_solutions_sub(l, t = 3, p = 5):
    
        b = []
        constraints = []
        print 'generating constraints ...'
        
        freq = [0,0,0]
        ag_set = []
        a_set = []
        
        
        for a in itertools.product(range(2), repeat = l):
            i = sage.misc.prandom.choice(range(p))
            if (i <= t):
                a_set.append(a)
                for g in itertools.product(range(2), repeat = l):
                    ag_set.append((a,g))
        
        ag_set_len = len(ag_set)
            
        print 'a_set = ',a_set
                    
        for t in itertools.product(range(2), repeat = 2*l):
            cur_row = [0 for i in range(ag_set_len)]
            b.append(1)
            for a in a_set:
                cur_row[ag_set.index((a, get_at(t,a,l)))] = 1
            constraints.append(cur_row)
        
        distr = [[0 for j in range(ag_set_len)] for i in range(3)]        
        present = [0,0,0]
        
        for a in a_set:
            y = sage.misc.prandom.choice(range(3))
            present[y] = 1
            for g in itertools.product(range(2), repeat = l):
                distr[y][ag_set.index((a,zeros))] = 1
                
                
        if sum(present[y] for y in range(3)) < 3:
            print 'Unluckly choice. Aborting execution...'
                
        for y in range(3):
            constraints.append(distr[y])
            b.append(1)
                
        m_constr = matrix(QQ,constraints)
        print_matrix(m_constr)
        right_side = vector(QQ, b)
        
        try:
            v = m_constr\right_side   
            print 'solution = ',v
            print 'len(solution) = ',len(v)
            nz = sum(1 for x in v if abs(x) > 0)
            print 'non-zeros ', nz
        except Exception as e:
            print 'Caught exception ',e
            
        

my_eps = QQ(1/10000)

In [66]:
import time 
   

for l in range(3, 4):
    for step in range(5):
        t0 = time.time()
        print 'finding a solution for l=',l
        zeros = tuple((0 for i in range(l)))

        try:
            q = test_linear_solutions_sub(l, 4, 5)

        finally:
            print 'time =', time.time() - t0,'\n'



finding a solution for l= 3
generating constraints ...
a_set =  [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
[1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 